# PCA on traits

In [3]:
import sys
import os
import torch
import pandas as pd
sys.path.append('.')
sys.path.append('..')

from utils.pca_utils import L2MeanScaler, MeanScaler,compute_pca, plot_variance_explained
from plots import plot_pc

## Configuration

In [2]:
# Configuration - Change these parameters for different models/datasets
base_dir = "/workspace/llama-3.3-70b"
type = "traits_240"
dir = f"{base_dir}/{type}"
model_name = "Llama-3.3-70B"
layer = 40

## Load vectors

In [5]:
# load all vectors from vectors
vector_dir = f"{dir}/vectors"

# iterate through each .pt file in the directory
vectors = {}
for file in os.listdir(vector_dir):
    if file.endswith(".pt"):
        vectors[file.replace(".pt", "")] = torch.load(os.path.join(vector_dir, file))

print(f"Found {len(vectors.keys())} traits with vectors")

Found 240 traits with vectors


In [6]:
# load default vectors
default_vectors = torch.load(f"{base_dir}/roles_240/default_vectors.pt")

In [7]:
print(vectors['zealous'].keys())
print(default_vectors.keys())
print(default_vectors['activations'].keys())

dict_keys(['pos_neg', 'pos_neg_50', 'pos_default', 'pos_default_50', 'pos_70', 'pos_40_70'])
dict_keys(['activations', 'metadata'])
dict_keys(['pos_1', 'default_1', 'all_1'])


## PCA

In [8]:
# load in stats
results_subdir = model_name.lower()
stats = pd.read_csv(f"./results/{results_subdir}/pos_neg.csv", index_col='trait')
print(stats.loc['zealous']['large_diff_count'])

1179.0


In [9]:
# PCA on pos_neg_50 but filter out traits with large_diff_count < 10
filtered_pos_neg_50_traits = []
filtered_pos_neg_50 = []

for trait, vector in vectors.items():
    if stats.loc[trait]['large_diff_count'] >= 10:
        filtered_pos_neg_50_traits.append(trait)
        filtered_pos_neg_50.append(vector['pos_neg_50'])
    else:
        print(f"Skipping {trait} because large_diff_count is {stats.loc[trait]['large_diff_count']}")

print(len(filtered_pos_neg_50_traits))

240


In [10]:
float_stack_vectors = torch.stack(filtered_pos_neg_50).float()
print(float_stack_vectors.shape)


torch.Size([240, 80, 8192])


In [11]:
# float_stack_vectors is (n_samples, n_layers, hidden_dims)
scaler = L2MeanScaler()

pca_transformed, variance_explained, n_components, pca, scaler = compute_pca(
    float_stack_vectors, 
    layer=layer, 
    scaler=scaler
)

PCA fitted with 240 components
Cumulative variance for first 5 components: [0.23312911 0.37244537 0.46423283 0.53920364 0.6000553 ]

PCA Analysis Results:
Elbow point at component: 2
Dimensions for 70% variance: 9
Dimensions for 80% variance: 16
Dimensions for 90% variance: 35
Dimensions for 95% variance: 61


In [12]:
# save pca
results = {}

results['layer'] = layer
results['traits'] = {
    'pos_neg_50': filtered_pos_neg_50_traits
}
results['vectors'] = {
    'pos_neg_50': filtered_pos_neg_50,
}
results['pca_transformed'] = pca_transformed
results['variance_explained'] = variance_explained
results['n_components'] = n_components
results['pca'] = pca
results['scaler'] = scaler

pca_dir = f"{dir}/pca"
os.makedirs(pca_dir, exist_ok=True)
torch.save(results, f"{pca_dir}/layer{layer}_normalized_pos-neg50.pt")

## Plots

In [13]:
plot_dir = f"/root/git/plots/{model_name.lower()}/traits"
os.makedirs(plot_dir, exist_ok=True)

normalized = True
if normalized:
    pca_results = torch.load(f"{dir}/pca/layer{layer}_normalized_pos-neg50.pt", weights_only=False)
else:
    pca_results = torch.load(f"{dir}/pca/layer{layer}_pos-neg50.pt", weights_only=False)


In [21]:
# flip PCs if needed
to_flip = [1]

for pc in to_flip:
    pca_results['pca'].components_[pc - 1] *= -1
    pca_results['pca_transformed'][:, pc - 1] *= -1
torch.save(pca_results, f"{dir}/pca/layer{layer}_normalized_pos-neg50.pt")

In [15]:
default_vectors = torch.load(f"{base_dir}/roles_240/default_vectors.pt")

In [22]:
# get default activationa nd project into PCA space
assistant_layer_activation = default_vectors['activations']['default_1'][layer, :].float().reshape(1, -1)

asst_scaled = pca_results['scaler'].transform(assistant_layer_activation)
asst_projected = pca_results['pca'].transform(asst_scaled)

assistant_layer_activation = assistant_layer_activation.numpy()

In [17]:
trait_labels = [trait.replace('_', ' ').title() for trait in pca_results['traits']['pos_neg_50']]

In [23]:
if normalized:
    subtitle = f"{model_name.replace('-', ' ')}, Layer {layer} - Mean-Centered and L2 Normalized Vectors"
else:
    subtitle = f"{model_name.replace('-', ' ')}, Layer {layer} - Shared Question Set, Mean-Centered and Scaled Vectors"

for i in range(6):
    fig = plot_pc(
        pca_results=pca_results,
        trait_labels=trait_labels,
        layer=layer,
        pc_component=i,
        assistant_activation=default_vectors['activations']['default_1'],
        assistant_projection=asst_projected[0],
        subtitle=subtitle,
        scaled=not normalized,
    )
    fig.show()

    if not normalized:
        fig.write_html(f"{plot_dir}/pc{i+1}.html")
    else:
        fig.write_html(f"{plot_dir}/pc{i+1}_normalized.html")

In [19]:
var_fig = plot_variance_explained(
    pca_results['variance_explained'],
    title="Variance Explained by Trait Role PC Components",
    subtitle=f"{model_name.replace('-', ' ')}, Layer {layer} - {len(trait_labels)} Total Components",
    max_components=50
)
var_fig.show()
var_fig.write_html(f"{plot_dir}/variance_explained.html")